# Build UI

In [23]:
import ipywidgets as widgets

### Button with a Value

In [24]:
class ButtonWithValue(widgets.Button):
    def __init__(self, *args, **kwargs):
        self.value = kwargs['value']
        kwargs.pop('value', None)
        super(ButtonWithValue, self).__init__(*args, **kwargs)

### Chromatic Number

In [25]:
import numpy as np
import networkx as nx
from qiskit import Aer
from qiskit.quantum_info import Pauli
from qiskit.opflow import PauliSumOp
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import ADAM


def get_operator(G):
    """Construct the cost Hamiltonian for the maximum independent set of a 
    a graph defined as:
        0.5 \sum_{i=nodes} Z_i + A/4 \sum_{ij = edges} Z_i x Z_j - Z_i - Z_j.
    Args:
        class graph (networkx): the graph in networkx.
    Returns:
        dict: dictionary of pauli Z matrices and its position.
    """
    
    A = 2.6
    num_nodes = G.number_of_nodes()
 
    pauli_list = []
    
    for i in range(num_nodes):
        x_p = np.zeros(num_nodes, dtype=bool)
        z_p = np.zeros(num_nodes, dtype=bool)
        z_p[i] = True 
        pauli_list.append([1/2,Pauli((z_p,x_p))])
        
    for pair in G.edges:
        x_p = np.zeros(num_nodes, dtype=bool)
        z_p = np.zeros(num_nodes, dtype=bool)
        z_p[pair[0]] = True
        z_p[pair[1]] = True
        pauli_list.append([A/4, Pauli((z_p,x_p))])
        
    for pair in G.edges:
        x_p = np.zeros(num_nodes, dtype=bool)
        z_p = np.zeros(num_nodes, dtype=bool)
        z_p[pair[0]] = True
        pauli_list.append([-A/4, Pauli((z_p,x_p))])
        
    for pair in G.edges:
        x_p = np.zeros(num_nodes, dtype=bool)
        z_p = np.zeros(num_nodes, dtype=bool)
        z_p[pair[1]] = True
        pauli_list.append([-A/4, Pauli((z_p,x_p))])
        
    pauli_list = [(pauli[1].to_label(), pauli[0]) for pauli in pauli_list]
    
    return PauliSumOp.from_list(pauli_list)


def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    n = int(np.log2(state_vector.shape[0]))
    k = np.argmax(np.abs(state_vector))
    x = np.zeros(n)
    for i in range(n):
       x[i] = k % 2
       k >>= 1
    return x

def max_clique(G):
    """Compute the QAOA algorithm and find the maximum independent set 
    of the graph G.
    Args:
        class graph (networkx): the graph in networkx.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    qubit_op = get_operator(G)
    optimizer = ADAM()
    qaoa = QAOA(optimizer, quantum_instance=Aer.get_backend('statevector_simulator'))

    result = qaoa.compute_minimum_eigenvalue(qubit_op)
                
    return sample_most_likely(result.eigenstate)


def chromatic_number(G):
    """Compute the chromatic number of a graph G.
    Args:
        class graph(networkx): the graph in networkx.
    Returns:
        int: integer ranging from 1 to number of nodes.  
    """
    color = 1
    while bool(G.edges):
        clique = max_clique(G)
        for i in range(len(clique)):
            if clique[i] == 1:
                G.remove_node(i)
        G=nx.convert_node_labels_to_integers(G) #relabel nodes starting from 0
        color += 1
    return color 

In [26]:
import retworkx as rx
import networkx as nx 
from retworkx import visualization
from qiskit.visualization import dag_drawer

### Graph Panel

In [27]:
def reset_game():
    g = nx.generators.gnm_random_graph(n = 5, m = 8)  
    graph = rx.networkx_converter(g)
    colors_dict = dict()

    for i in range(graph.num_nodes()):
        colors_dict[i] = "lightgray"
    
    return g, graph, colors_dict 

In [28]:
# retworkx 
g, graph, colors_dict = reset_game()

def node_colors(node):
    style = {
        'color' : colors_dict[node], 'fillcolor' : colors_dict[node], 'style' : 'filled'
    }
    return style

In [29]:
graph_text = widgets.HTML(r"<h1 class = 'title'> QColor </h1>")
graph_img = widgets.Output(layout = {'width' : '100%'})

graph_img.clear_output()
graph_img.layout = {
    "border": "1px solid #000",
    "padding": "2px 2px 2px 20%",
    "margin" : "2px", 
}
with graph_img:
    display(visualization.graphviz_draw(graph, node_attr_fn = node_colors)) 


In [30]:
def is_valid_color(graph, colors):
    # for each edge in graph 
    edges = list(graph.edge_list())
    
    for edge in edges:
        u = edge[0]
        v = edge[1]
        
        # both should be colored atleast
        if colors[u] != "lightgray" and colors[v] != "lightgray":
            if colors[u] == colors[v]:
                return False 
    return True
    

In [31]:
def complete_color(graph, colors):
    # for each edge in graph 
    edges = list(graph.edge_list())
    correct = 0
    for edge in edges:
        u = edge[0]
        v = edge[1]
        
        # both should be colored atleast
        if colors[u] != "lightgray" and colors[v] != "lightgray":
            if colors[u] != colors[v]:
                correct+=1 
    return correct == len(edges)
    

### Text box for interaction

In [32]:
def get_interaction_box(text):
    box = widgets.HTML("<h3> "+ text + " </h3>")
    box.layout = {
        "border": "1px solid #000",
        "padding": "5px 5px 5px 20px",
        "margin" : "3px",
        "overflow": "auto"
    }
    return box

In [33]:
interaction_box = get_interaction_box("Hey there!")

In [34]:
left_panel = widgets.VBox([graph_text, graph_img, interaction_box], layout = {"width" : "60%"})

In [35]:
display(left_panel)

### Callbacks for buttons

In [36]:
color_change = {
    "selected_color" : None
}

In [37]:
# each button on click performs different functions 
import time 
def color_callback(btn):
    color_val = btn.value 
    print("Value is :",color_val)
    color_change['selected_color'] = color_val

def vertex_callback(btn):
    
    # update the color of the button 
    
    # get the node id of the button
    node_id = int(btn.value)
    
    # now color it 
    nd_color = color_change['selected_color']
    
    invalid = False 
    
    if nd_color!="reset":
        # check whether we can color it and 
        # graph will remain a valid coloring 
        
        prev_color = colors_dict[node_id]
        
        # try to change the color 
        colors_dict[node_id] = nd_color
        
        
        if not is_valid_color(graph, colors_dict):
            # you need to alert the user and 
            # revert this color 
            text = "Sorry, invalid coloring!"
            print(text)
            colors_dict[node_id] = prev_color
            btn.style.button_color = '#ffdcd1'
            time.sleep(0.5)
            btn.style.button_color = '#feefff'
            invalid = True 
        else:
            btn.style.button_color = nd_color
    else:
        colors_dict[node_id] = 'lightgray'
        btn.style.button_color = 'lightgray'
    
    print("Dict :",colors_dict)
    
    children = main_panel.children[0].children
    
    head = children[0]
    out_panel = children[1]
    text_box = children[2]
    
    if not invalid:
        out_panel.clear_output()
        with out_panel:
            display(visualization.graphviz_draw(graph, node_attr_fn = node_colors))

    if complete_color(graph, colors_dict):
        text_box = get_interaction_box("You have won!")
        text_box.style.font_color = 'orange'
        
        main_panel.children[0].children = [head, out_panel, text_box]
        reset_game()

### Make the right box

In [38]:
vertex_text = widgets.HTML(r"<h2 class = 'title'> Vertices </h2>", layout = {"margin":"5px", "padding":"5px 5px 5px 30%"})
table_layout =  {
                "width" : "100%",
                "padding": "5px",
                "grid_template_columns": "repeat(3, 33%)",
            }

vertices = widgets.GridBox(
        layout = table_layout
)

children = []
for i in range(graph.num_nodes()):
    b = ButtonWithValue(description = str(i), value = str(i), layout = {"width" : "50px","border":"1px solid #000", "margin" : "5px", "height" : "50px"} )
    b.style.button_color = '#feefff'
    b.style.font_weight = 'bold'
    b.on_click(vertex_callback)
    children.append(b)
    
vertices.children = children

vertex_box = widgets.VBox([vertex_text, vertices], layout = dict(
        width="100%", display="flex", align_items="stretch", flex_flow="column"
    ))
display(vertex_box)

### Color Box

In [39]:
chrom_number = chromatic_number(g)

In [40]:
chro_number = widgets.HTML(r"<h2 class = 'title'> Chromatic Number  |  "+str(chrom_number) +"</h2>", layout = {"margin":"5px", "padding":"5px 5px 5px 0px"})

color_text = widgets.HTML(r"<h2 class = 'title'> Colors </h2>", layout = {"margin":"5px", "padding":"5px 5px 5px 30%"})
table_layout =  {
                "width" : "100%",
                "padding": "5px",
                "grid_template_columns": "repeat(3, 33%)",
            }
colors = widgets.GridBox(
        layout = table_layout
)

children = []
color = ['red', 'blue', 'yellow', 'green', 'gray','cyan','lightgreen']
import random
random.shuffle(color)


for i in range(chrom_number):
    b = ButtonWithValue(value = color[i], layout = {"width" : "50px", "margin" : "5px", "height" : "50px"} )
    b.style.button_color = color[i]
    b.style.font_weight = 'bold'
    b.on_click(color_callback)
    children.append(b)

reset = ButtonWithValue( value = "reset", icon = 'caret-down' ,layout = {"width" : "50px", "margin" : "5px", "height" : "50px"} )
reset.style.font_weight = 'bold'
reset.on_click(color_callback)
children.append(reset)
colors.children = children

color_box = widgets.VBox([chro_number, color_text, colors], layout = dict(
        width="100%", align_items="stretch", flex_flow="column"
    ))
display(color_box)

In [41]:
right_panel = widgets.VBox([vertex_box, color_box], layout = { "width" : "40%", "margin":"10px 10px 10px 30px"})

In [42]:
main_panel = widgets.HBox([left_panel, right_panel], layout = { "flex_flow" :"row", "width" : "100%"})

# Q-Color!

In [43]:
display(main_panel)

Value is : green
Dict : {0: 'lightgray', 1: 'lightgray', 2: 'lightgray', 3: 'green', 4: 'lightgray'}
Value is : green
Sorry, invalid coloring!
Dict : {0: 'lightgray', 1: 'lightgray', 2: 'lightgray', 3: 'green', 4: 'lightgray'}
Value is : cyan
Dict : {0: 'lightgray', 1: 'lightgray', 2: 'lightgray', 3: 'green', 4: 'cyan'}
Value is : red
Dict : {0: 'lightgray', 1: 'lightgray', 2: 'red', 3: 'green', 4: 'cyan'}
Value is : green
Value is : cyan
Dict : {0: 'cyan', 1: 'lightgray', 2: 'red', 3: 'green', 4: 'cyan'}
Value is : reset
Dict : {0: 'cyan', 1: 'lightgray', 2: 'red', 3: 'green', 4: 'lightgray'}
Value is : red
Sorry, invalid coloring!
Dict : {0: 'cyan', 1: 'lightgray', 2: 'red', 3: 'green', 4: 'lightgray'}
Value is : red
Dict : {0: 'cyan', 1: 'red', 2: 'red', 3: 'green', 4: 'lightgray'}
Value is : cyan
Dict : {0: 'cyan', 1: 'red', 2: 'red', 3: 'green', 4: 'cyan'}
Value is : reset
Dict : {0: 'cyan', 1: 'red', 2: 'red', 3: 'lightgray', 4: 'cyan'}
Value is : reset
Dict : {0: 'cyan', 1: 'lig